In [1]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import sklearn.decomposition


fname = '../data/100_unrelated.csv'
subjectids = np.loadtxt(fname, dtype=np.int)
yeo_order = list(sio.loadmat("../data/yeo_RS7_N374.mat", squeeze_me=True, struct_as_record=False)['yeoOrder']-1)
nSubj = len(subjectids)
tasks = ['rfMRI_REST1_LR', 'rfMRI_REST1_RL', 'rfMRI_REST2_LR',
         'rfMRI_REST2_RL', 'tfMRI_EMOTION_LR', 'tfMRI_EMOTION_RL',
         'tfMRI_GAMBLING_LR', 'tfMRI_GAMBLING_RL', 'tfMRI_LANGUAGE_LR',
         'tfMRI_LANGUAGE_RL', 'tfMRI_MOTOR_LR', 'tfMRI_MOTOR_RL',
         'tfMRI_RELATIONAL_LR', 'tfMRI_RELATIONAL_RL', 'tfMRI_SOCIAL_LR',
         'tfMRI_SOCIAL_RL', 'tfMRI_WM_LR', 'tfMRI_WM_RL']

M = {}
for task in tasks:
    masterFC_dir = '../data/results_SIFT2'
    restingstatename = 'fMRI/' + task + '/FC/FC_glasser_subc_GS_bp_z.mat'
    task_matrices = []
    for subject in subjectids:
        filename = masterFC_dir + '/' + str(subject) + '/' + restingstatename
        mat = sio.loadmat(filename, squeeze_me=True, struct_as_record=False)
        A_orig = np.array(mat['FC'])
        # yeo ordering
        A_orig = A_orig[np.ix_(yeo_order, yeo_order)]
        np.fill_diagonal(A_orig, 0)
        task_matrices.append(A_orig)
    M[task] = np.array(task_matrices)


In [2]:
task_dict = {}
task_dict["Rest"] = np.concatenate((M['rfMRI_REST1_LR'],M['rfMRI_REST1_RL']))
task_dict["Emotion"] = np.concatenate((M['tfMRI_EMOTION_LR'],M['tfMRI_EMOTION_RL']))
task_dict["Gambling"] = np.concatenate((M['tfMRI_GAMBLING_LR'], M['tfMRI_GAMBLING_RL']))
task_dict["Language"] = np.concatenate((M['tfMRI_LANGUAGE_LR'], M['tfMRI_LANGUAGE_RL']))
task_dict["Motor"] = np.concatenate((M['tfMRI_MOTOR_LR'], M['tfMRI_MOTOR_RL']))
task_dict["Relational"] = np.concatenate(( M['tfMRI_RELATIONAL_LR'], M['tfMRI_RELATIONAL_RL']))
task_dict["Social"] = np.concatenate((M['tfMRI_SOCIAL_LR'], M['tfMRI_SOCIAL_RL']))
task_dict["WM"] = np.concatenate((M['tfMRI_WM_LR'], M['tfMRI_WM_RL']))
del M

In [3]:
all_FC = np.concatenate(tuple(task_dict.values()))
del task_dict

## Deep Learning

In [4]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils import data
from sklearn.preprocessing import StandardScaler

# GPU is available? If so, we use it.
use_cuda = torch.cuda.is_available() 
device = torch.device("cuda:0" if use_cuda else "cpu")
if use_cuda:
    print("GPU detected. Will use GPU for training!")
    torch.backends.cudnn.benchmark = True
else:
    print("No GPU detected. Will use CPU for training.")

GPU detected. Will use GPU for training!


In [5]:
labels = torch.tensor(np.repeat(np.arange(0,8),nSubj*2), dtype=torch.long)
indices = np.random.permutation(labels.shape[0])
train_idx = indices[:int(0.6*labels.shape[0])]
val_idx = indices[int(0.6*labels.shape[0]):int(0.8*labels.shape[0])]
test_idx = indices[int(0.8*labels.shape[0]):]

In [6]:
train_mean = np.mean(all_FC[train_idx])
train_std = np.std(all_FC[train_idx])
train_data = torch.FloatTensor((all_FC[train_idx] - train_mean) / train_std)
val_data = torch.FloatTensor((all_FC[val_idx] - train_mean) / train_std)
test_data = torch.FloatTensor((all_FC[test_idx] - train_mean) / train_std)

In [7]:
train_data = train_data.view(train_data.shape[0], -1, train_data.shape[1], train_data.shape[2])
val_data = val_data.view(val_data.shape[0], -1, val_data.shape[1], val_data.shape[2])
test_data = test_data.view(test_data.shape[0], -1, test_data.shape[1], test_data.shape[2])

In [8]:
train_dataset = data.TensorDataset(train_data,labels[train_idx]) # create your datset
val_dataset = data.TensorDataset(val_data,labels[val_idx]) # create your datset
test_dataset = data.TensorDataset(test_data,labels[test_idx]) # create your datset

train_loader = data.DataLoader(train_dataset, batch_size=80) # create your dataloader
val_loader = data.DataLoader(val_dataset, batch_size=80) # create your dataloader
test_loader = data.DataLoader(test_dataset, batch_size=80) # create your dataloader


## NN Architecture

In [9]:
del all_FC
output_size = 8
max_epochs = 200
n_epochs_stop = 5

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 12, kernel_size=5)
        self.conv2_bn = nn.BatchNorm2d(12)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(18252, 128)
        self.fc2 = nn.Linear(128, 8)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), (3,3)))
        x = F.relu(F.max_pool2d(self.conv2_bn(self.conv2(x)), (3,3)))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
model = Net()


In [ ]:
loss_fn = nn.CrossEntropyLoss()
opt = optim.SGD(model.parameters(), lr=0.001)
if use_cuda:
    model = model.cuda()
print(model)

In [ ]:
load_model = False
if load_model:
    the_model = Net()
    the_model.load_state_dict(torch.load("lr0005"))

In [ ]:
history = {}
history['loss'] = []
history['val_loss'] = []
history['acc'] = []
history['val_acc'] = []
min_val_loss = np.Inf
early_stop = False
# Loop over epochs


for epoch in range(max_epochs):
    # Training
    model.train()
    train_loss         = 0.0
    num_train_correct  = 0
    num_train_examples = 0
    for local_batch, local_labels in train_loader:
        # Transfer to GPU
        if use_cuda:
            local_batch, local_labels = local_batch.to(device), local_labels.to(device)



        opt.zero_grad()
        output = model(local_batch)
        loss = loss_fn(output, local_labels)
        loss.backward()
        opt.step()

        train_loss += loss.data.item() * local_batch.size(0)
        num_train_correct  += (torch.max(output, 1)[1] == local_labels).sum().item()
        num_train_examples += local_batch.shape[0]

        train_acc = num_train_correct / num_train_examples
        train_loss  = train_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss       = 0.0
    num_val_correct  = 0
    num_val_examples = 0
    with torch.set_grad_enabled(False):
        for local_batch, local_labels in val_loader:
            # Transfer to GPU
            if use_cuda:
                local_batch, local_labels = local_batch.to(device), local_labels.to(device)
            output = model(local_batch)
            loss = loss_fn(output, local_labels)

            val_loss += loss.data.item() * local_batch.size(0)
            num_val_correct  += (torch.max(output, 1)[1] == local_labels).sum().item()
            num_val_examples += local_batch.shape[0]

        val_acc  = num_val_correct / num_val_examples
        val_loss = val_loss / len(val_loader.dataset)

        if val_loss < min_val_loss:
            epochs_no_improve = 0
            min_val_loss = val_loss
        else:
            epochs_no_improve += 1
  # Check early stopping condition
        if epochs_no_improve == n_epochs_stop:
            early_stop = print('Early stopping!')
            break

        
    print(f'Epoch {epoch+1}/{max_epochs}, train loss: {train_loss:5.4f}, train acc: {train_acc:5.3f}, val loss: {val_loss:5.3f}, val acc: {val_acc:5.3f}')
    
    
    history['loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['acc'].append(train_acc)
    history['val_acc'].append(val_acc)
    
    if early_stop:
        print("Stopped")
        break

In [ ]:
local_batch.device

In [ ]:
torch.save(model.state_dict(), "lr1e3_f5_sgd_yeoorder_190comps")

In [ ]:
model.eval()
num_correct  = 0
num_examples = 0
with torch.set_grad_enabled(False):
    for local_batch, local_labels in test_loader:
        # Transfer to GPU
        if use_cuda:
            local_batch, local_labels = local_batch.to(device), local_labels.to(device)
        output = model(local_batch)
        num_correct  += (torch.max(output, 1)[1] == local_labels).sum().item()
        num_examples += local_batch.shape[0]
            
    test_acc  = num_correct / num_examples

In [ ]:
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history['acc'])
plt.plot(history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
test_acc